# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,57.02,45,75,15.01,CA,1714949034
1,1,hovd,48.0056,91.6419,57.22,17,2,7.92,MN,1714949546
2,2,linhares,-19.3911,-40.0722,73.65,94,12,3.76,BR,1714949546
3,3,grytviken,-54.2811,-36.5092,37.18,87,70,9.48,GS,1714949546
4,4,edinburgh of the seven seas,-37.0676,-12.3116,55.90,63,48,30.94,SH,1714949546


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
%%capture --no-display

# Configure the map plot
map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="CartoLight",
    frame_width=800,
    frame_height=600,
    size = "Humidity",
    scale = 1,
)

# Display the map
map_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
idealcity_df = city_data_df.loc[(city_data_df['Max Temp'] <= 78) & (city_data_df['Max Temp'] >= 60)
                                 & (city_data_df['Humidity'] <= 74) & (city_data_df['Humidity'] <= 60)]


# Drop any rows with null values
idealcity_df = ideal_city_df.dropna(how="any")

# Display sample data
idealcity_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,carnarvon,-24.8667,113.6333,67.48,51,0,17.34,AU,1714949549
53,53,loei,17.4167,101.5000,65.57,54,0,1.99,TH,1714949552
62,62,moose jaw,50.4001,-105.5344,70.05,26,79,25.32,CA,1714949554
65,65,saint-pierre,-21.3393,55.4781,71.28,60,0,6.91,RE,1714949554
66,66,rice lake,45.5061,-91.7382,67.68,32,0,5.75,US,1714949554


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealcity_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
31,carnarvon,AU,-24.8667,113.6333,51,
53,loei,TH,17.4167,101.5000,54,
62,moose jaw,CA,50.4001,-105.5344,26,
65,saint-pierre,RE,-21.3393,55.4781,60,
66,rice lake,US,45.5061,-91.7382,32,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories": categories,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
carnarvon - nearest hotel: No hotel found
loei - nearest hotel: ภูเรือธารา
moose jaw - nearest hotel: Temple Gardens Mineral Spa Resort
saint-pierre - nearest hotel: Tropic Hotel
rice lake - nearest hotel: Paul Bunyan Resort
essaouira - nearest hotel: Essaouira Wind Palace
oudtshoorn - nearest hotel: The Pearl
lompoc - nearest hotel: Red Roof Inn Lompoc
gebeit - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
vernon - nearest hotel: Okanagan Royal Park Inn
prince george - nearest hotel: Travelodge by Wyndham Prince George
camalu - nearest hotel: Margaritaville
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
peace river - nearest hotel: New Ridge Inn
plettenberg bay - nearest hotel: Milkwood Manor
fort mcmurray - nearest hotel: Nomad Hotel and Suites
letlhakeng - nearest hotel: No hotel found
dadukou - nearest hotel: 锦江之星酒店
wenceslau braz - nearest hotel: No hotel found
taradale - nearest hotel: Ali Almajidi
oceansid

,City,Country,Lat,Lng,Humidity,Hotel Name
31,carnarvon,AU,-24.8667,113.6333,51,No hotel found
53,loei,TH,17.4167,101.5000,54,ภูเรือธารา
62,moose jaw,CA,50.4001,-105.5344,26,Temple Gardens Mineral Spa Resort
65,saint-pierre,RE,-21.3393,55.4781,60,Tropic Hotel
66,rice lake,US,45.5061,-91.7382,32,Paul Bunyan Resort
73,essaouira,MA,31.5125,-9.7700,60,Essaouira Wind Palace
96,oudtshoorn,ZA,-33.5907,22.2014,20,The Pearl
109,lompoc,US,34.6391,-120.4579,59,Red Roof Inn Lompoc
170,gebeit,SD,21.0667,36.3167,21,No hotel found
176,port alfred,ZA,-33.5906,26.8910,57,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="CartoLight",
    frame_width=800,
    frame_height=600,
    size = "Humidity",
    scale = 1,
    hover_cols=['Hotel Name', 'Country']
)
# Display the map
map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)